# Backtesting with backtrader library
Notes:  
Timestamp must be in the exact format specified for datafeed   **Timestamp needs to be first column index**  
Params for datafeed need to be specified as tuple of tuples  
Within strategy data is accessed via self.datas[0] for ex: self.datas[0].high_delta[0] would get you the first timeperiod's high_delta  
The current data has already happened and cannot be used to execute an order (ex. you cannot look at the current close price and simultaneously buy the current close, the order will be executed at open instead), orders will be executed on the following day.

In [8]:
import backtrader as bt
import datetime
import pandas as pd

## Define Strategies and Data utils

In [9]:
class BasicStrategy(bt.Strategy):
    def __init__(self):
        # To keep track of pending orders
        self.order = None
        print('***The limit sells for this strategy are valid for 1 day***')
        
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        elif order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)
            
            self.bar_executed = len(self)
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        
        # set no pending order
        self.order = None


    def next(self):
        # STRATEGY 
        data = self.datas[0]
        self.log(f'Current Portfolio Value : {self.broker.get_value()}')
        
        # cancel if there is an order pending, this strategy should have 1 working order per day
        if self.order:
            self.log('ORDER CANCELLED')
            self.cancel(self.order)
        
        # Check if we are in the market
        if not self.position:
            # BUY
            try:
                self.size = int(self.broker.get_cash() / self.datas[0].open[1])
            except:
                print('Size Exception. If at the end of data, ignore.')
            # invest if prediction looks good
            self.log(f'MARKET BUY CREATE {self.size} shares at next open, current close price: {data.close[0]}')
            self.buy(size=self.size) # market order buys at next open                      
                
        else:
            # place sell order at predicted high if predicted high is greater than current close price
            # TODO: Make prediction and close a filter in the class constructor (more optimal)
            if data.prediction[0] >= data.close[0]:
                self.log(f'LIMIT SELL CREATE {self.size} shares at {data.prediction[0]}')  
                self.sell(exectype=bt.Order.Limit,
                             price=data.prediction[0],
                             valid=data.datetime.date(0) + datetime.timedelta(days=2),
                             size=self.size)
        # if prediction is less than current value sell at open (ASAP)
            else:
                self.log('MARKET SELL CREATE. PREDICTION < CURRENT CLOSE')
                self.sell(size=self.size)
        

In [10]:
class BuyAndHold(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

      
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def nextstart(self):
        # Buy all the available cash
        size = int(self.broker.get_cash() / self.datas[0].open[1])
        self.buy(size=size)

    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print(f'Stop price: {self.datas[0].close[0]}')
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))

In [11]:
def prepare_data(data, fromdate, todate, filepath):
    """Prepare data for backtrader datafeed object
        Returns prepared data filepath and params for the GenericCSVData class, also returns columns used"""
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    # start param setup for backtrader
    start = data['timestamp'].iloc[0]
    end = data['timestamp'].iloc[-1]
    from_to = [(start.year, start.month, start.day), (end.year, end.month, end.day)]
    # Backtrader string format
    data['timestamp'] = data['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    # backtrader data feed class needs a file path, this could be a temp file that's constantly overwritten
    data.to_csv(filepath)
    
    starting_params = [
    ('fromdate', datetime.datetime(*from_to[0])),
    ('todate', datetime.datetime(*from_to[1])),
    ('nullvalue', 0.0),
    ('dtformat', ('%Y-%m-%d %H:%M:%S')),
    ('tmformat', ('%H:%M:%S')),
    ('datetime', 1)]
    # skip nonfeatures (timestamp)
    cols = data.columns[1:]
    # get column position for each indicator and add to starting params list
    i = 2 # starting index since others are reserved
    for indicator in cols:
        starting_params.append((indicator, i))
        i+=1
    final_params = tuple(starting_params)

    return filepath, final_params, cols 



## Import Data and Run Backtest

In [12]:
import os

In [19]:
def capstone_data_prep(pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000):
    """Main loop, preps data and executes backtrader on all stocks in the prediction folder"""
    ticker_backtesting_dict = {}
    for pred_file in os.listdir(f'../../data/ticker_predictions/{pred_folder_name}'):
        # collect data
        ticker_name = pred_file.split('_')[0]
        pred_df = pd.read_csv(f'../../data/ticker_predictions/{pred_folder_name}/{pred_file}')
        ticker_df = pd.read_csv(ticker_df_filepath + f'/{ticker_name}_full_data.csv')
        
        # make timestamp in column position 0, SPECIFIC TO FULL DATA IN FILE 
        ticker_df = ticker_df.rename({'reportperiod':'timestamp'}, axis=1)
        ticker_cols = list(ticker_df.columns)
        ticker_cols[5] = 'ts_cpy'
        ticker_cols[0] = 'timestamp'
        ticker_df.columns = ticker_cols
        ticker_df['timestamp'] = ticker_df['ts_cpy']
        # merge prediction data with full data, on key = timestamp
        ticker_df = ticker_df.merge(pred_df.loc[:, ['timestamp', pred_col_name]], on='timestamp')
        # new prediction column name = "prediction", important because backtester strategy looks for this column name
        ticker_df = ticker_df.rename({pred_col_name:'prediction'}, axis=1)
        # remove all non-prediction rows in prediction field
        ticker_df = ticker_df[ticker_df.prediction > 0]
        # skip nonfeatures, keep timestamp, SPECIFIC TO FULL DATA IN FILE 
        nonfeatures = ticker_df.columns[1:7]
        features = [col for col in ticker_df.columns if col not in nonfeatures]
        # get index of start and end dates for trading
        ticker_df = ticker_df[features]
        ticker_df.columns = [c.lower() for c in ticker_df.columns]
        idx1 = ticker_df.prediction[ticker_df.prediction > 0].index[0]
        idx2 = ticker_df.prediction[ticker_df.prediction > 0].index[-1]
        start_date = pred_df.timestamp[idx1]
        end_date  = pred_df.timestamp[idx2]
        
        # Prepare data for backtrader
        prep_data = prepare_data(ticker_df.copy(), start_date, end_date, 'temp/prep_data.csv')
        class DataFeed(bt.feeds.GenericCSVData):
            lines = tuple(prep_data[2])
            params = prep_data[1]
        
        print(f"--------Ticker Name-------: {ticker_name}")
        ##### BUY AND HOLD #######
        print('BUY AND HOLD')
        # initialize everything and run strategy
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        #cerebro.addsizer(bt.sizers.FixedSize, stake=10)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BuyAndHold)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        buy_hold = cerebro.run()
        buy_hold_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(buy_hold[0].analyzers.sharpe.get_analysis())
        print(buy_hold[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(buy_hold[0].analyzers.returns.get_analysis())

        #### MAIN STRATEGY #####
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BasicStrategy)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        back = cerebro.run()
        strat_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(back[0].analyzers.sharpe.get_analysis())
        print(back[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(back[0].analyzers.returns.get_analysis())
        print(f"For {ticker_name}, return was {buy_hold_ret*100}% for buy and hold vs {strat_ret*100}%  for strategy")
        print('\n'*5)

        ticker_backtesting_dict[ticker_name] = cerebro
        
    return ticker_backtesting_dict

In [21]:
# pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000

cerebro_dict = capstone_data_prep('linreg_preds', 'prediction', '../../data/ticker_data')
cerebro_dict

--------Ticker Name-------: UNH
BUY AND HOLD
Starting Portfolio Value: 100000
2019-09-18, BUY EXECUTED, 230.80
Stop price: 462.6199951171875
ROI:        100.38%
Final Portfolio Value: 200378.05656433105

STRATEGY INFO:
OrderedDict([('sharperatio', 0.38544087346000877)])
AutoOrderedDict([('len', 0), ('drawdown', 0.0), ('moneydown', 0.0), ('max', AutoOrderedDict([('len', 70), ('drawdown', 36.15894922151894), ('moneydown', 47824.84867858882)]))])
OrderedDict([('rtot', 0.6950356790452741), ('ravg', 0.0012823536513750444), ('rnorm', 0.3814768666517792), ('rnorm100', 38.14768666517792)])
Starting Portfolio Value: 100000
***The limit sells for this strategy are valid for 1 day***
2019-09-17, Current Portfolio Value : 100000.0
2019-09-17, MARKET BUY CREATE 433 shares at next open, current close price: 232.5200042724609
2019-09-18, BUY EXECUTED, 230.80
2019-09-18, Current Portfolio Value : 99917.72894287111
2019-09-18, LIMIT SELL CREATE 433 shares at 241.0939114132067
2019-09-19, Current Portfo

2020-02-28, Current Portfolio Value : 102507.29084011103
2020-02-28, LIMIT SELL CREATE 401 shares at 270.5867813626968
2020-03-02, Current Portfolio Value : 109785.43227382685
2020-03-02, LIMIT SELL CREATE 401 shares at 273.4955349392972
2020-03-03, SELL EXECUTED, 275.35
2020-03-03, Current Portfolio Value : 110683.68059536006
2020-03-03, MARKET BUY CREATE 381 shares at next open, current close price: 261.3999938964844
2020-03-04, BUY EXECUTED, 290.00
2020-03-04, Current Portfolio Value : 110462.70571132685
2020-03-04, MARKET SELL CREATE. PREDICTION < CURRENT CLOSE
2020-03-05, SELL EXECUTED, 281.60
2020-03-05, Current Portfolio Value : 107483.28292079951
2020-03-05, MARKET BUY CREATE 390 shares at next open, current close price: 282.9599914550781
2020-03-06, BUY EXECUTED, 275.42
2020-03-06, Current Portfolio Value : 110778.77577968623
2020-03-06, LIMIT SELL CREATE 390 shares at 302.0399608833212
2020-03-09, Current Portfolio Value : 106711.07863613154
2020-03-09, LIMIT SELL CREATE 390 

2020-09-01, Current Portfolio Value : 129125.00156399402
2020-09-01, LIMIT SELL CREATE 412 shares at 327.4357747883883
2020-09-02, Current Portfolio Value : 132177.9230727831
2020-09-02, LIMIT SELL CREATE 412 shares at 327.1990453104413
2020-09-03, Current Portfolio Value : 130525.81162258777
2020-09-03, LIMIT SELL CREATE 412 shares at 337.6284931924451
2020-09-04, Current Portfolio Value : 128783.04709622059
2020-09-04, LIMIT SELL CREATE 412 shares at 333.08956394148663
2020-09-08, Current Portfolio Value : 126826.04709622059
2020-09-08, LIMIT SELL CREATE 412 shares at 330.1951896945299
2020-09-09, Current Portfolio Value : 128791.2825698534
2020-09-09, LIMIT SELL CREATE 412 shares at 324.3439918199844
2020-09-10, Current Portfolio Value : 125734.24961086902
2020-09-10, LIMIT SELL CREATE 412 shares at 329.83652097281424
2020-09-11, Current Portfolio Value : 124448.81162258777
2020-09-11, LIMIT SELL CREATE 412 shares at 322.67435306435783
2020-09-14, Current Portfolio Value : 127369.89

2021-02-24, Current Portfolio Value : 130832.87419830612
2021-02-24, LIMIT SELL CREATE 393 shares at 343.3814188283747
2021-02-25, Current Portfolio Value : 129520.2556375151
2021-02-25, LIMIT SELL CREATE 393 shares at 346.7215290195038
2021-02-26, Current Portfolio Value : 130836.80803619674
2021-02-26, LIMIT SELL CREATE 393 shares at 340.43564482291674
2021-03-01, Current Portfolio Value : 131791.79515777878
2021-03-01, LIMIT SELL CREATE 393 shares at 344.68164228732815
2021-03-02, Current Portfolio Value : 131787.87331329635
2021-03-02, LIMIT SELL CREATE 393 shares at 346.689390399355
2021-03-03, Current Portfolio Value : 131092.2556375151
2021-03-03, LIMIT SELL CREATE 393 shares at 345.2451644659387
2021-03-04, Current Portfolio Value : 131461.67659698773
2021-03-04, LIMIT SELL CREATE 393 shares at 343.17156412410304
2021-03-05, SELL EXECUTED, 343.17
2021-03-05, Current Portfolio Value : 135140.77225723292
2021-03-05, MARKET BUY CREATE 389 shares at next open, current close price: 

2021-08-11, Current Portfolio Value : 156748.42359190035
2021-08-11, LIMIT SELL CREATE 384 shares at 427.3174808208598
2021-08-12, Current Portfolio Value : 156064.90406065035
2021-08-12, LIMIT SELL CREATE 384 shares at 423.3509376788274
2021-08-13, Current Portfolio Value : 157489.55249815035
2021-08-13, LIMIT SELL CREATE 384 shares at 420.7335862462198
2021-08-16, Current Portfolio Value : 160384.90406065035
2021-08-16, LIMIT SELL CREATE 384 shares at 424.6435655669352
2021-08-17, Current Portfolio Value : 162139.78687315035
2021-08-17, LIMIT SELL CREATE 384 shares at 436.636113114694
2021-08-18, Current Portfolio Value : 160427.15015440038
2021-08-18, LIMIT SELL CREATE 384 shares at 437.521013437088
2021-08-19, Current Portfolio Value : 164486.03296690035
2021-08-19, LIMIT SELL CREATE 384 shares at 428.3031655832161
2021-08-20, SELL EXECUTED, 428.35
2021-08-20, Current Portfolio Value : 164662.66968565035
2021-08-20, MARKET BUY CREATE 382 shares at next open, current close price: 42

...

2020-09-03, BUY EXECUTED, 229.27
2020-09-03, Current Portfolio Value : 207269.27029050852
2020-09-03, LIMIT SELL CREATE 953 shares at 232.42895268970008
2020-09-04, Current Portfolio Value : 204362.61738218332
2020-09-04, LIMIT SELL CREATE 953 shares at 224.36412618113715
2020-09-08, Current Portfolio Value : 193317.35087217353
2020-09-08, LIMIT SELL CREATE 953 shares at 217.73979330635703
2020-09-09, Current Portfolio Value : 201541.7309838679
2020-09-09, MARKET SELL CREATE. PREDICTION < CURRENT CLOSE
2020-09-10, SELL EXECUTED, 213.40
2020-09-10, Current Portfolio Value : 203552.56156553296
2020-09-10, MARKET BUY CREATE 982 shares at next open, current close price: 205.3699951171875
2020-09-11, BUY EXECUTED, 207.20
2020-09-11, Current Portfolio Value : 200439.62336362866
2020-09-11, LIMIT SELL CREATE 982 shares at 220.1790241392434
2020-09-14, Current Portfolio Value : 201794.7881585505
2020-09-14, LIMIT SELL CREATE 982 shares at 214.1722460117772
2020-09-15, Current Portfolio Value :

2020-12-04, Current Portfolio Value : 205912.76813719433
2020-12-04, LIMIT SELL CREATE 960 shares at 219.0855580922364
2020-12-07, Current Portfolio Value : 205845.5611059443
2020-12-07, LIMIT SELL CREATE 960 shares at 218.57519736985037
2020-12-08, Current Portfolio Value : 207496.76227781933
2020-12-08, LIMIT SELL CREATE 960 shares at 218.88523674765267
2020-12-09, Current Portfolio Value : 203455.1704809443
2020-12-09, LIMIT SELL CREATE 960 shares at 220.97112104848563
2020-12-10, Current Portfolio Value : 202226.37165281927
2020-12-10, LIMIT SELL CREATE 960 shares at 217.5169670864144
2020-12-11, Current Portfolio Value : 204856.76227781933
2020-12-11, LIMIT SELL CREATE 960 shares at 216.0693103033582
2020-12-14, Current Portfolio Value : 205759.1646215693
2020-12-14, LIMIT SELL CREATE 960 shares at 218.3908618414109
2020-12-15, Current Portfolio Value : 205691.9722387568
2020-12-15, LIMIT SELL CREATE 960 shares at 220.89634294651165
2020-12-16, Current Portfolio Value : 210635.966

2021-04-07, BUY EXECUTED, 247.81
2021-04-07, Current Portfolio Value : 236876.70849911406
2021-04-07, LIMIT SELL CREATE 947 shares at 250.3166599479548
2021-04-08, SELL EXECUTED, 252.77
2021-04-08, Current Portfolio Value : 239594.6083251638
2021-04-08, MARKET BUY CREATE 947 shares at next open, current close price: 253.25
2021-04-09, BUY EXECUTED, 252.87
2021-04-09, Current Portfolio Value : 242416.6787292165
2021-04-09, MARKET SELL CREATE. PREDICTION < CURRENT CLOSE
2021-04-12, SELL EXECUTED, 254.71
2021-04-12, Current Portfolio Value : 241337.09930721947
2021-04-12, MARKET BUY CREATE 938 shares at next open, current close price: 255.91000366210935
2021-04-13, BUY EXECUTED, 257.26
2021-04-13, Current Portfolio Value : 242490.82098690697
2021-04-13, MARKET SELL CREATE. PREDICTION < CURRENT CLOSE
2021-04-14, SELL EXECUTED, 257.48
2021-04-14, Current Portfolio Value : 241543.460452239
2021-04-14, MARKET BUY CREATE 936 shares at next open, current close price: 255.58999633789065
2021-04-

2021-09-17, MARKET SELL CREATE. PREDICTION < CURRENT CLOSE
2021-09-20, SELL EXECUTED, 296.33
2021-09-20, Current Portfolio Value : 252295.8662536856
2021-09-20, MARKET BUY CREATE 853 shares at next open, current close price: 294.29998779296875
2021-09-21, BUY EXECUTED, 295.69
2021-09-21, Current Portfolio Value : 251536.68375856843
2021-09-21, MARKET SELL CREATE. PREDICTION < CURRENT CLOSE
2021-09-22, SELL EXECUTED, 296.73
2021-09-22, Current Portfolio Value : 253182.99354250397
2021-09-22, MARKET BUY CREATE 847 shares at next open, current close price: 298.5799865722656
2021-09-23, BUY EXECUTED, 298.85
2021-09-23, Current Portfolio Value : 253784.3563049551
2021-09-23, MARKET SELL CREATE. PREDICTION < CURRENT CLOSE
2021-09-24, SELL EXECUTED, 298.23
2021-09-24, Current Portfolio Value : 252657.85767824616
2021-09-24, MARKET BUY CREATE 853 shares at next open, current close price: 299.3500061035156
2021-09-27, BUY EXECUTED, 296.14
2021-09-27, Current Portfolio Value : 250977.4466369864


{'UNH': <backtrader.cerebro.Cerebro at 0x7f89207c7850>,
 'ADSK': <backtrader.cerebro.Cerebro at 0x7f8932a8aa90>,
 'XOM': <backtrader.cerebro.Cerebro at 0x7f8970fb1690>,
 'WAT': <backtrader.cerebro.Cerebro at 0x7f89512c7710>,
 'HD': <backtrader.cerebro.Cerebro at 0x7f89606fbb10>,
 'MSFT': <backtrader.cerebro.Cerebro at 0x7f89514c9b10>}

### Library additionally has plotting
Not sure how to reformat this, it's pretty ugly

In [25]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams.update({'font.size': 12}) 
cerebro_dict['UNH'].plot()

ImportError: cannot import name 'warnings' from 'matplotlib.dates' (/opt/anaconda3/lib/python3.7/site-packages/matplotlib/dates.py)

### Ways to Improve:

Increase strategy trading volume. There are certain periods where sell orders are not going through for multiple days in a row, meaning that the price is lower than anticipated by model for that period. For example, if the last sell order didn't go through, we can find a way to use that information to inform our next sell order and make the prediction lower or try a market sell. Maybe train a "bear" model, one that only knows pain and suffering of bear markets, and only produces pessimistic high price predictions.   

Smarter entry points. Currently, buy orders are being executed at market price at the open. Returns may increase if entry constraints are increased.  

Resizing orders. Orders for this strategy baseline are currently 100% in or 100% out of position. This has beneifits if you want less exposure to the long term movement of the stocks. Depending on the situation and strategy, we might want to reserve a cash position to expose ourselves less to short term movements.  